# Python Project

In [1]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
import requests, time
import plotly

In [2]:
# analize each thirds
# proportions for small, med and big delays

In [3]:
pd.set_option('display.max_columns', None)

df = pd.read_csv('airport_data.csv')

print(df.shape)
df.head()

(6489062, 32)


,MONTH,DAY_OF_WEEK,DEP_DELAY_NEW,DEP_DEL15,DEP_TIME_BLK,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,CARRIER_NAME,AIRPORT_FLIGHTS_MONTH,AIRLINE_FLIGHTS_MONTH,AIRLINE_AIRPORT_FLIGHTS_MONTH,AVG_MONTHLY_PASS_AIRPORT,AVG_MONTHLY_PASS_AIRLINE,FLT_ATTENDANTS_PER_PASS,GROUND_SERV_PER_PASS,PLANE_AGE,DEPARTING_AIRPORT,LATITUDE,LONGITUDE,PREVIOUS_AIRPORT,PRCP,SNOW,SNWD,TMAX,AWND
0,1,7,0.0,0,0800-0859,2,NaN,NaN,NaN,NaN,NaN,1,25,143,Southwest Airlines Co.,13056,107363,5873,1903352,13382999,0.000062,0.000099,8,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91
1,1,7,0.0,0,0700-0759,7,NaN,NaN,NaN,NaN,NaN,1,29,191,Delta Air Lines Inc.,13056,73508,1174,1903352,12460183,0.000144,0.000149,3,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91
2,1,7,0.0,0,0600-0659,7,NaN,NaN,NaN,NaN,NaN,1,27,199,Delta Air Lines Inc.,13056,73508,1174,1903352,12460183,0.000144,0.000149,18,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91
3,1,7,0.0,0,0600-0659,9,NaN,NaN,NaN,NaN,NaN,1,27,180,Delta Air Lines Inc.,13056,73508,1174,1903352,12460183,0.000144,0.000149,2,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91
4,1,7,0.0,0,0001-0559,7,NaN,NaN,NaN,NaN,NaN,1,10,182,Spirit Air Lines,13056,15023,1257,1903352,2688839,0.000009,0.000125,1,McCarran International,36.08,-115.152,NONE,0.0,0.0,0.0,65.0,2.91


In [4]:
df.isnull().sum().reset_index().head(32)

,index,0
0,MONTH,0
1,DAY_OF_WEEK,0
2,DEP_DELAY_NEW,0
3,DEP_DEL15,0
4,DEP_TIME_BLK,0
5,DISTANCE_GROUP,0
6,CARRIER_DELAY,5242063
7,WEATHER_DELAY,5242063
8,NAS_DELAY,5242063
9,SECURITY_DELAY,5242063


In [5]:
# Delay Flags ranges

cd = df[df['CARRIER_DELAY']>0].CARRIER_DELAY
wd = df[df['WEATHER_DELAY']>0].WEATHER_DELAY
nd = df[df['NAS_DELAY']>0].NAS_DELAY
sd = df[df['SECURITY_DELAY']>0].SECURITY_DELAY
ld = df[df['LATE_AIRCRAFT_DELAY']>0].LATE_AIRCRAFT_DELAY

print(f'Carrier Delay min:{cd.min()} max:{cd.max()}')
print(f'Weather Delay min:{wd.min()} max:{wd.max()}')
print(f'NAS Delay min:{nd.min()} max:{nd.max()}')
print(f'Security Delay min:{sd.min()} max:{sd.max()}')
print(f'Late Aircraft Delay min:{ld.min()} max:{ld.max()}')

Carrier Delay min:1.0 max:2695.0
Weather Delay min:1.0 max:1575.0
NAS Delay min:1.0 max:1711.0
Security Delay min:1.0 max:1078.0
Late Aircraft Delay min:1.0 max:2010.0


In [6]:
df.loc[df['CARRIER_DELAY']>0, 'CARRIER_DELAY'] = 1
df.loc[df['WEATHER_DELAY']>0, 'WEATHER_DELAY'] = 1
df.loc[df['NAS_DELAY']>0, 'NAS_DELAY'] = 1
df.loc[df['SECURITY_DELAY']>0, 'SECURITY_DELAY'] = 1
df.loc[df['LATE_AIRCRAFT_DELAY']>0, 'LATE_AIRCRAFT_DELAY'] = 1

df.loc[df['DEP_DEL15']==0, 'CARRIER_DELAY'] = 0
df.loc[df['DEP_DEL15']==0, 'WEATHER_DELAY'] = 0
df.loc[df['DEP_DEL15']==0, 'NAS_DELAY'] = 0
df.loc[df['DEP_DEL15']==0, 'SECURITY_DELAY'] = 0
df.loc[df['DEP_DEL15']==0, 'LATE_AIRCRAFT_DELAY'] = 0

In [7]:
# What is the percentage of delays?

df.DEP_DEL15.value_counts(normalize=True)

0    0.810856
1    0.189144
Name: DEP_DEL15, dtype: float64

In [8]:
# How are those delays distributed through causes?

print(df[df['DEP_DEL15']>0].CARRIER_DELAY.value_counts(normalize=True))
print(df[df['DEP_DEL15']>0].WEATHER_DELAY.value_counts(normalize=True))
print(df[df['DEP_DEL15']>0].NAS_DELAY.value_counts(normalize=True))
print(df[df['DEP_DEL15']>0].SECURITY_DELAY.value_counts(normalize=True))
print(df[df['DEP_DEL15']>0].LATE_AIRCRAFT_DELAY.value_counts(normalize=True))

# There can be more than one flag for a delay

1.0    0.568601
0.0    0.431399
Name: CARRIER_DELAY, dtype: float64
0.0    0.934077
1.0    0.065923
Name: WEATHER_DELAY, dtype: float64
0.0    0.587845
1.0    0.412155
Name: NAS_DELAY, dtype: float64
0.0    0.996604
1.0    0.003396
Name: SECURITY_DELAY, dtype: float64
1.0    0.618277
0.0    0.381723
Name: LATE_AIRCRAFT_DELAY, dtype: float64


In [9]:
for delay in ['CARRIER_DELAY', 'WEATHER_DELAY', 'LATE_AIRCRAFT_DELAY']:
    df.loc[(df[delay] == 1) & (df['DEP_DELAY_NEW'] > 15), f'SIZE_{delay}'] = f'SMALL_{delay}'
    df.loc[(df[delay] == 1) & (df['DEP_DELAY_NEW'] > 30), f'SIZE_{delay}'] = f'MEDIUM_{delay}'
    df.loc[(df[delay] == 1) & (df['DEP_DELAY_NEW'] > 60), f'SIZE_{delay}'] = f'BIG_{delay}'

df.loc[df['DEP_DELAY_NEW'] <= 15, f'SIZE_DELAY'] = f'NO_DELAY'
df.loc[df['DEP_DELAY_NEW'] > 15, f'SIZE_DELAY'] = f'SMALL_DELAY'
df.loc[df['DEP_DELAY_NEW'] > 30, f'SIZE_DELAY'] = f'MEDIUM_DELAY'
df.loc[df['DEP_DELAY_NEW'] > 60, f'SIZE_DELAY'] = f'BIG_DELAY'

In [10]:
df = pd.get_dummies(df, columns=['SIZE_CARRIER_DELAY'])
df = pd.get_dummies(df, columns=['SIZE_WEATHER_DELAY'])
df = pd.get_dummies(df, columns=['SIZE_LATE_AIRCRAFT_DELAY'])

df = df.fillna(0)

In [11]:
df[df['DEP_DEL15']==1].describe()

,MONTH,DAY_OF_WEEK,DEP_DELAY_NEW,DEP_DEL15,DISTANCE_GROUP,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,SEGMENT_NUMBER,CONCURRENT_FLIGHTS,NUMBER_OF_SEATS,AIRPORT_FLIGHTS_MONTH,AIRLINE_FLIGHTS_MONTH,AIRLINE_AIRPORT_FLIGHTS_MONTH,AVG_MONTHLY_PASS_AIRPORT,AVG_MONTHLY_PASS_AIRLINE,FLT_ATTENDANTS_PER_PASS,GROUND_SERV_PER_PASS,PLANE_AGE,LATITUDE,LONGITUDE,PRCP,SNOW,SNWD,TMAX,AWND,SIZE_CARRIER_DELAY_BIG_CARRIER_DELAY,SIZE_CARRIER_DELAY_MEDIUM_CARRIER_DELAY,SIZE_CARRIER_DELAY_SMALL_CARRIER_DELAY,SIZE_WEATHER_DELAY_BIG_WEATHER_DELAY,SIZE_WEATHER_DELAY_MEDIUM_WEATHER_DELAY,SIZE_WEATHER_DELAY_SMALL_WEATHER_DELAY,SIZE_LATE_AIRCRAFT_DELAY_BIG_LATE_AIRCRAFT_DELAY,SIZE_LATE_AIRCRAFT_DELAY_MEDIUM_LATE_AIRCRAFT_DELAY,SIZE_LATE_AIRCRAFT_DELAY_SMALL_LATE_AIRCRAFT_DELAY
count,1.227368e+06,1.227368e+06,1.227368e+06,1227368.0,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06,1.227368e+06
mean,6.473086e+00,3.934775e+00,6.841584e+01,1.0,3.901447e+00,4.521586e-01,5.242275e-02,3.277509e-01,2.700901e-03,4.916618e-01,3.474653e+00,2.823882e+01,1.348790e+02,1.317401e+04,6.321175e+04,3.579936e+03,1.645376e+06,7.828103e+06,9.715877e-05,1.339499e-04,1.162142e+01,3.671139e+01,-9.325033e+01,1.607532e-01,6.451472e-02,1.309154e-01,7.112888e+01,8.721881e+00,1.854603e-01,1.582476e-01,1.028616e-01,2.879414e-02,1.459790e-02,8.501118e-03,2.223465e-01,1.709601e-01,9.405655e-02
std,3.329377e+00,1.992616e+00,8.684447e+01,0.0,2.379533e+00,4.977061e-01,2.228781e-01,4.693937e-01,5.189999e-02,4.999307e-01,1.771501e+00,2.129837e+01,4.648374e+01,8.731558e+03,3.510032e+04,4.094565e+03,1.095574e+06,5.043405e+06,8.588854e-05,4.811397e-05,6.765633e+00,5.250422e+00,1.681165e+01,4.325139e-01,4.883145e-01,8.562936e-01,1.924260e+01,3.778255e+00,3.886707e-01,3.649731e-01,3.037781e-01,1.672276e-01,1.199367e-01,9.180880e-02,4.158229e-01,3.764743e-01,2.919075e-01
min,1.000000e+00,1.000000e+00,1.500000e+01,1.0,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,4.400000e+01,1.100000e+03,5.582000e+03,1.000000e+00,7.047600e+04,4.737940e+05,0.000000e+00,7.134695e-06,0.000000e+00,1.844000e+01,-1.593460e+02,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.000000e+00,2.000000e+00,2.400000e+01,1.0,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,1.100000e+01,9.900000e+01,5.847000e+03,2.488600e+04,7.250000e+02,7.325950e+05,2.688839e+06,3.419267e-05,9.889412e-05,5.000000e+00,3.343600e+01,-1.048800e+02,0.000000e+00,0.000000e+00,0.000000e+00,5.900000e+01,6.040000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.000000e+00,4.000000e+00,4.100000e+01,1.0,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00,2.300000e+01,1.430000e+02,1.196500e+04,7.047300e+04,2.566000e+03,1.581456e+06,8.501631e+06,6.178236e-05,1.238227e-04,1.200000e+01,3.761900e+01,-8.790600e+01,0.000000e+00,0.000000e+00,0.000000e+00,7.500000e+01,8.050000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,9.000000e+00,6.000000e+00,8.000000e+01,1.0,5.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,5.000000e+00,3.900000e+01,1.720000e+02,1.854300e+04,8.718300e+04,5.016000e+03,2.006675e+06,1.246018e+07,1.441659e-04,1.772872e-04,1.700000e+01,4.077900e+01,-8.093600e+01,1.000000e-01,0.000000e+00,0.000000e+00,8.700000